In [11]:
import numpy as np
import utils.prompt as prUt
import torch
from utils.base_classes import Points



In [5]:
# Get initial segmentation
from classes.SAMMed2DClass_unstable import SAMMed2DInferer
import utils.analysis as anUt
import utils.prompt as prUt
from utils.image import read_im_gt
from utils.interactivity import iterate_2d

# Obtain model, image, gt
device = 'cuda'
sammed2d_checkpoint_path = "/home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth"
sammed2d_inferer = SAMMed2DInferer(sammed2d_checkpoint_path, device)

img_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001_0000.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/Dataset350_AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
img, gt = read_im_gt(img_path, gt_path, 3)

# Obtain initial segmentation: Point propagation

seed = 11121
n_clicks = 5

# Get seed prompt and bounds
seed_point = prUt.get_seed_point(gt, n_clicks, seed)
slices_to_infer = np.where(np.any(gt, axis=(1,2)))[0]

segmentation, intial_prompt = prUt.point_propagation(sammed2d_inferer, img, seed_point, slices_to_infer, seed, n_clicks)
print(anUt.compute_dice(gt,segmentation))


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations


True
*******load /home/t722s/Desktop/UniversalModels/TrainedModels/sam-med2d_b.pth


Propagating up: 100%|██████████| 22/22 [00:01<00:00, 21.96it/s]


0.8177772392806715


In [17]:
# Iteratively improve
condition = 'dof'
dof_bound = 90
seed_sub = np.random.randint(10**5)
segmentation, dof, segmentations, prompts, max_fp_idxs = iterate_2d(sammed2d_inferer, img, gt, segmentation, intial_prompt, 
                                                                         condition = 'dof', init_dof = 5, dof_bound = dof_bound, seed = seed_sub, detailed = True)

0.8100428200388422
0.8168069928284744
0.8193945832127969
0.8209056779737632
0.8210853952797726
0.8184585818801077
0.805741004190195
0.806110206112157
degrees of freedom bound met; terminating with performance 0.806110206112157


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# def show_mask(mask, ax, random_color=False):
#     if random_color:
#         color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
#     else:
#         color = np.array([30/255, 144/255, 255/255, 0.6])

#     h, w = mask.shape[-2:]
#     mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     ax.imshow(mask_image)
# # Visualise
# plt.figure(figsize = (13,13))
# plt.imshow(max_fp_slice, cmap = 'gray')
# show_mask(slice_seg, plt.gca())
# # contour_rounded = np.round(contour).astype(int)
# # plt.scatter(contour_rounded[:,1], contour_rounded[:,0], c= 'green', s = 10)
# plt.scatter(scribble_coords[:,1], scribble_coords[:,0], c = 'red', s = 2)

In [14]:
# for seg, max_fp_idx in zip(segmentations, max_fp_idx):
#     slice_seg = seg[:, max_fp_idx]
#     max_fp_slice = gt[:, max_fp_idx]
#     # Visualise
#     plt.figure(figsize = (13,13))
#     plt.imshow(max_fp_slice, cmap = 'gray')
#     show_mask(slice_seg, plt.gca())
#     # contour_rounded = np.round(contour).astype(int)
#     # plt.scatter(contour_rounded[:,1], contour_rounded[:,0], c= 'green', s = 10)
#     plt.scatter(scribble_coords[:,1], scribble_coords[:,0], c = 'red', s = 2)

NameError: name 'max_fp_slices' is not defined